In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [164]:
data = pd.read_csv('data/train.csv')
data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
data = np.array(data)
m, n = data.shape
#np.random.shuffle(data)
print(data.shape)

data_train = data.T
Y_train = data_train[0]
X_train = data_train[1:788]
X_train = X_train / 255.

print(X_train.shape)

print(X_train[68][26])

(42000, 785)
(784, 42000)
0.11372549019607843


In [166]:
#test cell
np.random.rand(10, 784) - 0.5

array([[ 0.44296122, -0.26874216,  0.40286282, ..., -0.36006885,
         0.21323916,  0.02029577],
       [ 0.18108555, -0.16830678, -0.14032772, ...,  0.29373389,
         0.15528437, -0.11649232],
       [ 0.09081952,  0.26314337, -0.36365953, ...,  0.03835697,
        -0.25029494,  0.03217384],
       ...,
       [-0.05825293, -0.4390376 , -0.28364141, ...,  0.03611617,
         0.09206891,  0.2251336 ],
       [-0.02241949,  0.37670735,  0.07809907, ...,  0.40317593,
        -0.31622434,  0.16137909],
       [-0.18331352, -0.16294913, -0.49770343, ...,  0.43980724,
         0.05798771, -0.42616125]])

In [314]:
from contextlib import redirect_stdout

def create(X):
    c = 0
    for i in range(0, X.shape[0]):
        for j in range(0, X.shape[1]):
            c+=1
            if c%2==1:
                X[i][j] = -0.3
            else:
                X[i][j] = 0.3
    return X

def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    W1 = create(W1)
    W2 = create(W2)
    b1 = create(b1)
    b2 = create(b2)
    return W1, b1, W2, b2

def LReLU(Z):
    return np.where(Z > 0, Z, 0.01 * Z)

def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))

def ReLU(Z):
    return np.maximum(0, Z)

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    
    dZ2 = A2 - one_hot_Y
    dW2 = 1/m * dZ2.dot(A1.T)
    
    db2 = 1/m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * (Z1 > 0)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2



In [315]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):

    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [316]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.3, 100)

2.7711166694643907e-12
Iteration:  0
[1 1 1 ... 1 1 1] [1 0 1 ... 7 6 9]
0.11152380952380953
2.7000623958883807e-12
2.3590018827235326e-12
2.9487523534044158e-12
2.5579538487363607e-12
2.1955770534987096e-12
1.9895196601282805e-12
1.6413537196058314e-12
1.7053025658242404e-12
1.2931877790833823e-12
9.094947017729282e-13
Iteration:  10
[1 1 0 ... 1 1 1] [1 0 1 ... 7 6 9]
0.1144047619047619


In [313]:
import numpy as np

# Use genfromtxt to handle rows with a different number of columns
data = np.genfromtxt('out.txt', filling_values=np.nan)  # Use NaN to fill missing values

print(np.sum(data))

2.7711166694643907e-12


In [296]:
a = np.array([[-0.3, 0.3,  -0.3], [-0.3, 0.3,  -0.3]])
b = np.array([[0.4, 0.6], [0.64, 0.75],[0.65,-0.23]])
print(a.dot(b))

[[-0.123  0.114]
 [-0.123  0.114]]
